In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'd:\\ML-Project'

### Building our entity

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

### Building configuration manager

In [4]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path
        )

        return data_transformation_config

### Creating a Component

In [5]:
import os
from mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        train,test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted data into training and test sets.")

        logger.info(f"Training data shape: {train.shape}")
        logger.info(f"Test data shape: {test.shape}")


### **Creating a pipeline for data transformation**

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2023-09-26 11:31:50,209: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-26 11:31:50,225: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-26 11:31:50,241: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-26 11:31:50,241: INFO: common: created directory at: artifacts]
[2023-09-26 11:31:50,241: INFO: common: created directory at: artifacts/data_transformation]
[2023-09-26 11:31:50,313: INFO: 1063081258: Splitted data into training and test sets.]
[2023-09-26 11:31:50,321: INFO: 1063081258: Training data shape: (1199, 12)]
[2023-09-26 11:31:50,321: INFO: 1063081258: Test data shape: (400, 12)]
